## Support-Resistance Breakout Strategy

原則:  
- 選擇股價波動大且成交量大的標的  
- 定義 ```breakout rule```  
- 定義 ```exit/stop loss signal```  
- 用 cumulative return 計算每個標的的 returnm

策略:  
- ```breakout rule```: 20 period rolling max/min + 20 days rolling volume  
- ```exit/stop loss```: previous price +- 20 period ATR

### 1. 引入套件及 KPI functions:

In [44]:
import tejapi

In [61]:
tejapi.ApiConfig.api_key = "09vOL7Mk5yamPfvzSffVkl6iKb0fHJ"

In [62]:
info = tejapi.ApiConfig.info()

In [64]:
data = tejapi.get('TRAIL/TAGIN')

In [65]:
data

,coid,mdate,gin0,l0ng_ta,buy_l,buy_la,sell_l,sell_la,cash_l,cash_la,...,limit_b1,limit_b2,sbls_k,shorts_k,ap,alc,asc1,lmr,smr,tmr
None,,,,,,,,,,,,,,,,,,,,,
0,0050,2020-01-02 00:00:00+00:00,374,36513.62,20,1952.60,6,585.78,0,0.0,...,1006,68700,31990.95,195.4,97.6300,95.93,87.11,169.65,169.49,169.58
1,0050,2020-01-03 00:00:00+00:00,388,37907.60,30,2931.00,16,1563.20,0,0.0,...,1041,66950,21216.45,195.7,97.7000,96.07,87.23,169.41,169.73,169.55
2,0050,2020-01-06 00:00:00+00:00,381,36827.46,7,676.62,14,1353.24,0,0.0,...,1120,66950,33959.20,386.5,96.6600,96.08,87.45,167.23,172.36,169.44
3,0050,2020-01-07 00:00:00+00:00,410,39351.80,41,3935.18,12,1151.76,0,0.0,...,1158,66950,49379.95,287.6,95.9800,96.07,87.61,166.72,173.21,169.32
4,0050,2020-01-08 00:00:00+00:00,441,42133.14,51,4872.54,20,1910.80,0,0.0,...,1205,66050,53815.30,477.8,95.5400,96.01,87.84,166.05,174.49,169.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,00657,2020-10-23 00:00:00+00:00,0,0.00,0,0.00,0,0.00,0,0.0,...,32,432,0.00,0.0,29.2496,NaN,NaN,NaN,NaN,NaN
9996,00657,2020-10-26 00:00:00+00:00,0,0.00,0,0.00,0,0.00,0,0.0,...,32,432,0.00,0.0,29.1601,NaN,NaN,NaN,NaN,NaN
9997,00657,2020-10-27 00:00:00+00:00,0,0.00,0,0.00,0,0.00,0,0.0,...,32,432,0.00,0.0,29.0498,NaN,NaN,NaN,NaN,NaN


In [47]:
import numpy as np
import pandas as pd
import copy
import time

In [48]:
from alpha_vantage.timeseries import TimeSeries

In [149]:
def ATR(DF, n):
    df = DF.copy()
    df["H-L"] = abs(df["High"] - df["Low"]) 
    df["H-PC"] = abs(df["High"] - df["Close"].shift(1)) # 今天跳空上漲的情況
    df["L-PC"] = abs(df["Low"] - df["Close"].shift(1)) # 今天跳空下跌
    # 取三者裡的最大者，且不忽略 NaN
    df["TR"] = df[ ["H-L", "H-PC", "L-PC"] ].max(axis=1, skipna=False)
    df["ATR"] = df["TR"].rolling(n).mean() # rolling 的天數自訂，通常用 20
    # 刪除暫時的 columns
    df = df.drop( ["H-L", "H-PC", "L-PC"], axis=1)
    return df["ATR"]

In [150]:
def CAGR(DF):
    df = DF.copy()
    df['Daily Return'] = df['Adj Close'].pct_change() # first we need to calculate daily return
    df['Cumulative Return'] = (1 + df['Daily Return']).cumprod() # calculate cumulative return
    n = len(df) / 250*54 # assume there are 250 trading days in a year
    cagr = (df['Cumulative Return'][-1])**(1/n) - 1 # we need the latest data to calculate CAGR
    return cagr

注意: 上面 CAGR function 的 ```n``` 必須多除 ```54```，因為這個策略每五分鐘執行一次，而台股一天有四個半小時 (```270``` 分鐘)的交易時間  
，故總次數為 ```270 / 5 = 54```

In [ ]:
def Volatility(DF):
    df = DF.copy()
    df['Daily Return'] = df['Adj Close'].pct_change()
    vol = df['Daily Return'].std() * (250*54 ** 0.5) # converst daily volatility to annual volatility
    return vol

Same as ```Volatility```

In [29]:
def Sharp(DF, rf): # rf stands for "risk free rate"
    df = DF.copy()
    sr = (CAGR(df) - rf) / Volatility(df)
    return sr

In [30]:
def Max_DD(DF):
    df = DF.copy()
    df['Daily Return'] = df['Adj Close'].pct_change()
    df['Cumulative Return'] = (1 + df['Daily Return']).cumprod() # 這麼多天的總報酬
    df['Cumulative Rolling Max'] = df['Cumulative Return'].cummax() # ?

    df['Draw Down'] = df['Cumulative Rolling Max'] - df['Cumulative Return']
    df['Draw Down Percentage'] = df['Draw Down'] / df['Cumulative Rolling Max']

    max_dd = df['Draw Down Percentage'].max()
    return max_dd

### 2.1. 選擇交易標的:

先用一個股票拆解，之後再用迴圈取得多個股票:

In [95]:
key_path = r"C:\\Users\User\\Quant_Trading\\alpha_vantage\\api_key.txt"
ts = TimeSeries(key=open(key_path, 'r').read(), output_format='pandas')

In [96]:
data = ts.get_intraday(symbol='MSFT', interval='5min', outputsize='full')

In [97]:
type(data) # return tuple object

tuple

In [98]:
data

(                     1. open  2. high  3. low  4. close  5. volume
 date                                                              
 2021-01-28 20:00:00   236.60   236.62  236.42    236.42     3142.0
 2021-01-28 19:55:00   236.65   236.76  236.60    236.61     1710.0
 2021-01-28 19:50:00   236.30   236.75  236.30    236.75     1660.0
 2021-01-28 19:45:00   236.70   236.75  236.30    236.31     5428.0
 2021-01-28 19:40:00   236.45   236.45  236.41    236.41     1229.0
 ...                      ...      ...     ...       ...        ...
 2020-12-31 07:05:00   222.05   222.11  222.01    222.02     1752.0
 2020-12-31 06:55:00   222.35   222.35  222.35    222.35      111.0
 2020-12-31 05:15:00   222.54   222.68  222.54    222.68      538.0
 2020-12-31 04:55:00   222.45   222.45  222.45    222.45      241.0
 2020-12-31 04:20:00   221.90   221.90  221.85    221.85      600.0
 
 [3290 rows x 5 columns],
 {'1. Information': 'Intraday (5min) open, high, low, close prices and volume',
  '2. Sy

In [99]:
data[0] # 這裡的資料由最新到最就是由上往下，應該把它倒過來

,1. open,2. high,3. low,4. close,5. volume
date,,,,,
2021-01-28 20:00:00,236.60,236.62,236.42,236.42,3142.0
2021-01-28 19:55:00,236.65,236.76,236.60,236.61,1710.0
2021-01-28 19:50:00,236.30,236.75,236.30,236.75,1660.0
2021-01-28 19:45:00,236.70,236.75,236.30,236.31,5428.0
2021-01-28 19:40:00,236.45,236.45,236.41,236.41,1229.0
...,...,...,...,...,...
2020-12-31 07:05:00,222.05,222.11,222.01,222.02,1752.0
2020-12-31 06:55:00,222.35,222.35,222.35,222.35,111.0
2020-12-31 05:15:00,222.54,222.68,222.54,222.68,538.0


In [100]:
type(data[0]) # we only need this data frame

pandas.core.frame.DataFrame

In [101]:
data[0].iloc[::-1] # change the order of the dates

,1. open,2. high,3. low,4. close,5. volume
date,,,,,
2020-12-31 04:20:00,221.90,221.90,221.85,221.85,600.0
2020-12-31 04:55:00,222.45,222.45,222.45,222.45,241.0
2020-12-31 05:15:00,222.54,222.68,222.54,222.68,538.0
2020-12-31 06:55:00,222.35,222.35,222.35,222.35,111.0
2020-12-31 07:05:00,222.05,222.11,222.01,222.02,1752.0
...,...,...,...,...,...
2021-01-28 19:40:00,236.45,236.45,236.41,236.41,1229.0
2021-01-28 19:45:00,236.70,236.75,236.30,236.31,5428.0
2021-01-28 19:50:00,236.30,236.75,236.30,236.75,1660.0


In [106]:
data

(                     1. open  2. high  3. low  4. close  5. volume
 date                                                              
 2021-01-28 20:00:00   236.60   236.62  236.42    236.42     3142.0
 2021-01-28 19:55:00   236.65   236.76  236.60    236.61     1710.0
 2021-01-28 19:50:00   236.30   236.75  236.30    236.75     1660.0
 2021-01-28 19:45:00   236.70   236.75  236.30    236.31     5428.0
 2021-01-28 19:40:00   236.45   236.45  236.41    236.41     1229.0
 ...                      ...      ...     ...       ...        ...
 2020-12-31 07:05:00   222.05   222.11  222.01    222.02     1752.0
 2020-12-31 06:55:00   222.35   222.35  222.35    222.35      111.0
 2020-12-31 05:15:00   222.54   222.68  222.54    222.68      538.0
 2020-12-31 04:55:00   222.45   222.45  222.45    222.45      241.0
 2020-12-31 04:20:00   221.90   221.90  221.85    221.85      600.0
 
 [3290 rows x 5 columns],
 {'1. Information': 'Intraday (5min) open, high, low, close prices and volume',
  '2. Sy

In [119]:
df = pd.DataFrame(data[0])

In [120]:
df # 發現到 MSFT 的時間部正確，應該要改成美國股市開盤的時間

,Open,High,Low,Close,Volume
date,,,,,
2021-01-28 20:00:00,236.60,236.62,236.42,236.42,3142.0
2021-01-28 19:55:00,236.65,236.76,236.60,236.61,1710.0
2021-01-28 19:50:00,236.30,236.75,236.30,236.75,1660.0
2021-01-28 19:45:00,236.70,236.75,236.30,236.31,5428.0
2021-01-28 19:40:00,236.45,236.45,236.41,236.41,1229.0
...,...,...,...,...,...
2020-12-31 07:05:00,222.05,222.11,222.01,222.02,1752.0
2020-12-31 06:55:00,222.35,222.35,222.35,222.35,111.0
2020-12-31 05:15:00,222.54,222.68,222.54,222.68,538.0


In [121]:
df.columns = ['Open', 'High', 'Low', 'Close', 'Volume']

In [122]:
df = df.between_time('09:35', '16:00')

In [123]:
df

,Open,High,Low,Close,Volume
date,,,,,
2021-01-28 16:00:00,240.11,240.1200,238.6100,238.9300,1556311.0
2021-01-28 15:55:00,239.15,240.2600,238.7201,240.1200,1039693.0
2021-01-28 15:50:00,238.74,239.3500,238.2400,239.1300,612485.0
2021-01-28 15:45:00,239.36,239.4600,238.6600,238.7599,426632.0
2021-01-28 15:40:00,238.89,239.4900,238.5800,239.3600,372897.0
...,...,...,...,...,...
2020-12-31 09:55:00,221.97,222.3100,221.7400,221.8000,275213.0
2020-12-31 09:50:00,222.14,222.3737,221.9238,221.9600,247078.0
2020-12-31 09:45:00,222.22,222.5000,222.1200,222.1200,317844.0


### 2.2. 將上面的 code 用迴圈包起來:

In [131]:
tickers = ['MSFT', 'AAPL', 'AMZN', 'TSLA', 'AMD']

In [132]:
key_path = r"C:\\Users\User\\Quant_Trading\\alpha_vantage\\api_key.txt"
ts = TimeSeries(key=open(key_path, 'r').read(), output_format='pandas')

In [133]:
ohlc_intraday = {} # use a dict to store all the tickers' ohlc
api_call_count = 1 # track how many times we have done
start_time = time.time()

In [134]:
for ticker in tickers:
    data = ts.get_intraday(symbol=ticker, interval='5min', outputsize='full')[0]
    api_call_count += 1
    data.columns = ["Open", "High", "Low", "Close", "Volume"]

    data = data.iloc[::-1]
    data = data.between_time('09:35', '16:00')

    ohlc_intraday[ticker] = data # store particular stock's information

    if api_call_count == 5:
        api_call_count = 1
        time.sleep(60 - ((time.time() - start_time) % 60.0))

In [146]:
ohlc_intraday

{'MSFT':                        Open      High       Low     Close     Volume
 date                                                                
 2020-12-31 09:35:00  221.70  222.3200  221.7000  221.8100   671955.0
 2020-12-31 09:40:00  221.81  222.2781  221.4500  222.2149   391996.0
 2020-12-31 09:45:00  222.22  222.5000  222.1200  222.1200   317844.0
 2020-12-31 09:50:00  222.14  222.3737  221.9238  221.9600   247078.0
 2020-12-31 09:55:00  221.97  222.3100  221.7400  221.8000   275213.0
 ...                     ...       ...       ...       ...        ...
 2021-01-28 15:40:00  238.89  239.4900  238.5800  239.3600   372897.0
 2021-01-28 15:45:00  239.36  239.4600  238.6600  238.7599   426632.0
 2021-01-28 15:50:00  238.74  239.3500  238.2400  239.1300   612485.0
 2021-01-28 15:55:00  239.15  240.2600  238.7201  240.1200  1039693.0
 2021-01-28 16:00:00  240.11  240.1200  238.6100  238.9300  1556311.0
 
 [1482 rows x 5 columns],
 'AAPL':                          Open    High       L

In [137]:
tickers = ohlc_intraday.keys() # 如果在迴圈內有某些股票無資料，則應去掉。故更新過的 ohlc_intraday 的 key 重新定義 tickers

 ### 3. 建構好策略所需的技術指標資料:

第一步非常重要: 複製一份 ```ohlc_dict```。 因為 ```ohlc_dict``` 是用迴圈方式取得的，最好不要隨意更動。

In [138]:
ohlc_dict = copy.deepcopy(ohlc_intraday)

In [139]:
tickers_signal = {} # 用來放各個股票在所有時點的交易訊號 (buy, sell, hold)

In [141]:
tickers_ret = {} 

In [158]:
for ticker in tickers:
    print("Calculating ATR and rolling max price for: ", ticker)

    # get the technical data
    ohlc_dict[ticker]["ATR"] = ATR(ohlc_dict[ticker], 20)
    ohlc_dict[ticker]["Rolling_max_cp"] = ohlc_dict[ticker]["High"].rolling(20).max()
    ohlc_dict[ticker]["Rolling_min_cp"] = ohlc_dict[ticker]["Low"].rolling(20).min()
    ohlc_dict[ticker]["Rolling_max_vol"] = ohlc_dict[ticker]["Volume"].rolling(20).max()
    ohlc_dict[ticker].dropna(inplace=True)

    # set the two di"ct to original
    tickers_signal[ticker] = ""
    tickers_ret[ticker] = [0]


Calculating ATR and rolling max price for:  MSFT
Calculating ATR and rolling max price for:  AAPL
Calculating ATR and rolling max price for:  AMZN
Calculating ATR and rolling max price for:  TSLA
Calculating ATR and rolling max price for:  AMD


In [159]:
tickers_signal

{'MSFT': '', 'AAPL': '', 'AMZN': '', 'TSLA': '', 'AMD': ''}

In [160]:
tickers_ret

{'MSFT': [0], 'AAPL': [0], 'AMZN': [0], 'TSLA': [0], 'AMD': [0]}

In [161]:
ohlc_dict['MSFT']

,Open,High,Low,Close,Volume,ATR,Rolling_max_cp,Rolling_min_cp,Rolling_max_vol
date,,,,,,,,,
2020-12-31 11:15:00,220.5000,220.5200,220.2300,220.3900,201061.0,0.352175,222.5000,220.23,391996.0
2020-12-31 11:20:00,220.4000,220.6200,220.3600,220.5857,171990.0,0.323770,222.5000,220.23,317844.0
2020-12-31 11:25:00,220.5785,220.5900,220.4000,220.5000,85159.0,0.314270,222.3737,220.23,278368.0
2020-12-31 11:30:00,220.4950,220.5100,220.2500,220.2800,319535.0,0.304775,222.3100,220.23,319535.0
2020-12-31 11:35:00,220.2750,220.3353,220.1900,220.1952,241678.0,0.283540,221.8300,220.19,319535.0
...,...,...,...,...,...,...,...,...,...
2021-01-28 15:40:00,238.8900,239.4900,238.5800,239.3600,372897.0,0.716480,242.5500,238.58,619782.0
2021-01-28 15:45:00,239.3600,239.4600,238.6600,238.7599,426632.0,0.732980,242.4637,238.58,619782.0
2021-01-28 15:50:00,238.7400,239.3500,238.2400,239.1300,612485.0,0.755045,242.1350,238.24,619782.0
